In [9]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [14]:
import os 
from pinecone import Pinecone, ServerlessSpec 

index_name = "hybrid-search-langchain-pinecone"

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384, # Dimension of dense vector
        metric="dotproduct", #Sparse value is supported only for dot product
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [16]:
index = pc.Index(index_name)
index

d:\Code\Python\langchain-app\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Vector embeddings and sparse matrix
import os 
from langchain_huggingface import HuggingFaceEmbeddings 
from dotenv import load_dotenv 
load_dotenv()

os.environ["HF_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
from pinecone_text.sparse import BM25Encoder 

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [23]:
sentences = [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]
# TFIDF values on these sentences 
bm25_encoder.fit(sentences)

# Store values in JSON file 
bm25_encoder.dump("b25_values.json")

# Load value from JSON file 
bm25_encoder = BM25Encoder().load("b25_values.json")

100%|██████████| 3/3 [00:00<00:00, 38.74it/s]


In [24]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=bm25_encoder,
    index=index
)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x00000230DF50D720>, index=<pinecone.db_data.index.Index object at 0x00000230B2837F70>)

In [25]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:04<00:00,  4.62s/it]


In [27]:
retriever.invoke("Which city did I visit in 2022?")

[Document(metadata={'score': 0.490436673}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.355387896}, page_content='In 2023, I visited Paris'),
 Document(metadata={'score': 0.339708447}, page_content='In 2021, I visited New Orleans')]